In [1]:
import json
import time
import datetime
import dateutil.parser
from datetime import datetime
import pandas as pd

In [2]:
def isEndorsedByStaff(endorsements):
    for endorsement in endorsements:
        if 'role' in endorsement and ('professor' in endorsement['role'] or 'instructor' in endorsement['role'] or 'ta' in endorsement['role']):
            return True
    
def checkValidAnswer(post):
    return ('i_answer' in post['type']) or ('tag_endorse' in post and isEndorsedByStaff(post['tag_endorse']))

def getAnswerList(post):
    answerList = []
    if('children' in post):
        postAnswers = post['children']
        for postAnswer in postAnswers:
            #whoAnswered = ''
            answer = ''
            if 'type' in postAnswer and checkValidAnswer(postAnswer) and 'history' in postAnswer and 'subject' not in postAnswer['history']:
                #whoAnswered = postAnswer['type']
                #last_modified = postAnswer['history'][len(postAnswer['history']) - 1]
                last_modified = getLastModified(postAnswer)
                answer = last_modified['content']
                #answerToWhoAnswered = (answer, whoAnswered)
                answerList.append(answer)
    return answerList

def getLastModified(post):
    history = post['history']
    last_modified_answer = history[0]
    last_modified_datetime = dateutil.parser.parse(history[0]['created'])
    for i in range(0, len(history)):
        post_datetime = dateutil.parser.parse(history[i]['created'])
        if(post_datetime > last_modified_datetime):
            last_modified_datetime = post_date
            last_modified_answer = history[i]
            
    return last_modified_answer

def extractData(filename):
    with open(filename, 'r') as openfile:
        input = json.load(openfile)
        #print(input)
        df = pd.DataFrame(columns = ['Post', 'Sentence'])
        for i in range(0, len(input)):
            post = input[i]
            if 'history' in post:
                last_modified = getLastModified(post)
                if 'subject' in last_modified and 'content' in last_modified and '<img' not in last_modified['content']:
                    subject = last_modified['subject']
                    content = last_modified['content']
                    post_ID = post['nr']
                    answerList = getAnswerList(input[i])
                    df = df.append({'Post': post_ID, 'Sentence': subject + "." + content}, ignore_index = True)  
                    for i in range(0, len(answerList)):
                        df = df.append({'Post': post_ID, 'Sentence': answerList[i]}, ignore_index = True)

        return df
#     for i in range(0, len(df)):
#         print("Post: "+str(int(df.iloc[i]['Post'])))
#         print("Sentence: "+df.iloc[i]['Sentence'])


In [4]:
#usage
final_df = pd.concat([extractData("data/fall_22_nlp.json"), extractData("data/spring_22_nlp.json")])
print(final_df.to_string())

   Post                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

/var/folders/p8/fbqgjwlx683072z049myq84r0000gn/T/ipykernel_15786/1192936527.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Post': post_ID, 'Sentence': subject + "." + content}, ignore_index = True)
/var/folders/p8/fbqgjwlx683072z049myq84r0000gn/T/ipykernel_15786/1192936527.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Post': post_ID, 'Sentence': answerList[i]}, ignore_index = True)
/var/folders/p8/fbqgjwlx683072z049myq84r0000gn/T/ipykernel_15786/1192936527.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Post': post_ID, 'Sentence': subject + "." + content}, ignore_index = True)
/var/folders/p8/fbqgjwlx683072z049myq84r0000gn/T/ipykernel_15786/1192936527.py:5

In [12]:
#Convert to Lowercase
final_df["Sentence"] = final_df["Sentence"].str.lower()

In [15]:
from bs4 import BeautifulSoup

#Remove all HTML tags
final_df["Sentence"] = final_df["Sentence"].apply(lambda x: BeautifulSoup(str(x)).get_text())

In [18]:
import re

#Remove all URLs
final_df["Sentence"] = final_df["Sentence"].apply(lambda x: re.sub(r'\s*(https?://|www\.)+\S+(\s+|$)', " ", str(x), flags=re.UNICODE))

In [20]:
#Remove extra spaces
final_df["Sentence"] = final_df["Sentence"].apply(lambda x: re.sub(r"\s+", " ", str(x), flags=re.UNICODE))

In [22]:
import contractions

#Removing all contractions
def perform_contractions(series):
    series = series.apply(lambda x: contractions.fix(x))
    return series

x = perform_contractions(final_df["Sentence"])
final_df["Sentence"] = x

#Removing uppercase letters which might be introduced after removing contractions
final_df["Sentence"] = final_df["Sentence"].str.lower()

In [23]:
#final_df["Sentence"]

0     quiz. 9 - neural language models use word-embe...
1     yes, word embedding are not used to rain word2...
2     regarding random shuffling between epochs whil...
3                           not mandatory, but see @661
4     feature extraction and classifying using one m...
                            ...                        
19    since probabilities lie between 0 and 1, multi...
20    logistic regression robustness to correlated f...
21    generally yes. one should get rid of correlate...
22    emojis.are emojis considered to be part of nlp...
23    not sure if i understand the question. so plea...
Name: Sentence, Length: 93, dtype: object

In [26]:
#final_df

,Post,Sentence
0,689,quiz. 9 - neural language models use word-embe...
1,689,"yes, word embedding are not used to rain word2..."
2,663,regarding random shuffling between epochs whil...
3,663,"not mandatory, but see @661"
4,638,feature extraction and classifying using one m...
...,...,...
19,103,"since probabilities lie between 0 and 1, multi..."
20,54,logistic regression robustness to correlated f...
21,54,generally yes. one should get rid of correlate...
22,18,emojis.are emojis considered to be part of nlp...
